In [18]:
pid = os.getpid()
!kill -9 $pid

In [1]:
import numpy as np
import os
import os.path
import pandas as pd
pd.options.display.max_colwidth = 100
import cv2
import tensorflow as tf
import pickle
import copy
from PIL import Image, ImageDraw

# VARIABLES

In [2]:
model_path = '/home/acarlier/code/OrnithoScope/saved_models/model_efficientDet1.tflite'
input_csv_path = '/home/acarlier/code/OrnithoScope/data/input.csv'
columns_names = ["split_value", "file_path", "label",
                      "x_min", "y_min", "empty_1", "empty_2",
                      "x_max", "y_max", "empty_3"]
output_images_directory = '/home/acarlier/code/data_ornitho/output_images/test_pickle/'
comparaison_images_directory = '/home/acarlier/code/data_ornitho/comparaison_verite_terrain/comparaison_efficientDet1/'


# FUNCTIONS

In [3]:
#@title Load the trained TFLite model and define some visualization functions

def get_classes_from_input():
    input = pd.read_csv(input_csv_path, names = columns_names)
    return input["label"].unique().tolist()

# Load the labels into a list
classes = get_classes_from_input()
# Define a list of colors for visualization
COLORS = np.random.randint(0, 255, size=(len(classes), 3), dtype=np.uint8)

def preprocess_image(image_path, input_size):
  """Preprocess the input image to feed to the TFLite model"""
  img = tf.io.read_file(image_path)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  resized_img = tf.cast(resized_img, dtype=tf.uint8)
  #print(tf.shape(resized_img))
  return resized_img, original_image

def preprocess_images(image_paths, input_size):
  """Preprocess the input image to feed to the TFLite model"""
  img = tf.io.read_file(image_paths[0])
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  resized_img = tf.cast(resized_img, dtype=tf.uint8)
  
  resized_imgs = resized_img
  
  for i in range(1,len(image_paths)):
    img = tf.io.read_file(image_paths[i])
    img = tf.io.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.uint8)
    resized_img = tf.image.resize(img, input_size)
    resized_img = resized_img[tf.newaxis, :]
    resized_img = tf.cast(resized_img, dtype=tf.uint8)
    
    resized_imgs = tf.concat([resized_imgs, resized_img], axis=0)
    #print(tf.shape(resized_imgs))
    
  return resized_imgs


def detect_objects(interpreter, image, threshold):
  """Returns a list of detection results, each a dictionary of object info."""

  signature_fn = interpreter.get_signature_runner()
  #print("avant modèle")
  # Feed the input image to the model
  output = signature_fn(images=image)
  #interpreter.set_tensor(0, image)
  #interpreter.invoke()
  #print("après modèle")
  
  # Get all outputs from the model
  count = int(np.squeeze(output['output_0']))
  scores = np.squeeze(output['output_1'])
  classes = np.squeeze(output['output_2'])
  boxes = np.squeeze(output['output_3'])

  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
  return results


def run_odt_and_draw_results(image_path, interpreter, threshold=0.5):
  """Run object detection on the input image and draw the detection results"""
  # Load the input shape required by the model
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']
  #print("interpreter")
  
  # Load the input image and preprocess it
  preprocessed_image, _ = preprocess_image(
      image_path,
      (input_height, input_width)
    )
  #print("preprocessing")
  
  # Run object detection on the input image
  results = detect_objects(interpreter, preprocessed_image, threshold=threshold)
  #print("detection")
  
  if 0:
    # Plot the detection results on the input image
    original_image_np = original_image.numpy().astype(np.uint8)
    for obj in results:
      # Convert the object bounding box from relative coordinates to absolute
      # coordinates based on the original image resolution
      ymin, xmin, ymax, xmax = obj['bounding_box']
      xmin = int(xmin * original_image_np.shape[1])
      xmax = int(xmax * original_image_np.shape[1])
      ymin = int(ymin * original_image_np.shape[0])
      ymax = int(ymax * original_image_np.shape[0])

      # Find the class index of the current object
      class_id = int(obj['class_id'])

      # Draw the bounding box and label on the image
      color = [int(c) for c in COLORS[class_id]]
      cv2.rectangle(original_image_np, (xmin, ymin), (xmax, ymax), color, 2)
      # Make adjustments to make the label visible for all objects
      y = ymin - 15 if ymin - 15 > 15 else ymin + 15
      label = "{}: {:.0f}%".format(classes[class_id], obj['score'] * 100)
      cv2.putText(original_image_np, label, (xmin, y),
          cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Return the final image
    original_uint8 = original_image_np.astype(np.uint8)
  original_uint8 = None  
  return original_uint8, results

# Génération des prédictions

In [4]:
def get_test_images():
    input = pd.read_csv(input_csv_path, names = columns_names)
    test_dataset = input.loc[input['split_value'] == 'TEST']
    list_image_test = test_dataset["file_path"].unique().tolist() # return all unique file_path to test, no prediction on two same image
    return list_image_test
loi = get_test_images()
print(loi[0])
print(len(loi))

/home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_2021-03-01_09/20210301-090002_(13.0).jpg
3516


In [5]:
def prediction_test_dataset():
    results = []
    list_of_images = get_test_images()
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    #list_of_images = ['/home/acarlier/code/test/2021-06-11-13-44-43.jpg']
    # Load the TFLite model
    print(len(list_of_images),' to predict')
    for img in list_of_images:
        if list_of_images.index(img)%100 == 0:
            print(list_of_images.index(img))
        img_name = img.split('/')[-1]
        file_path = '{}{}'.format(output_images_directory, img_name)
        if os.path.exists(file_path):
            list_of_images.remove(img)
            #print(len(list_of_images),' remaining')
        else:
            INPUT_IMAGE_URL = img #@param {type:"string"}
            DETECTION_THRESHOLD = 0.3 #@param {type:"number"}
            TEMP_FILE = img
            FILE_NAME = TEMP_FILE.split("/")[-1]
            #print('test1.2 : ok')
            #!wget -q -O $TEMP_FILE $INPUT_IMAGE_URL
            #!wget -q -O $INPUT_IMAGE_URL
            #print('test1.3 : ok')
            try:
                with open(TEMP_FILE) as file:
                    #im = Image.open(TEMP_FILE)
                    #im.thumbnail((512, 512), Image.ANTIALIAS)
                    #im.save(TEMP_FILE, 'JPEG')
                    # Run inference and draw detection result on the local copy of the original file
                    detection_result_image, result = run_odt_and_draw_results(
                        TEMP_FILE,
                        interpreter,
                        threshold=DETECTION_THRESHOLD
                    )
                    results.append(result)
                    # Show the detection result
                    #Image.fromarray(detection_result_image)
                    #result = Image.fromarray(detection_result_image)
                    #result.save("{}{}".format(output_images_directory, FILE_NAME), 'JPEG')
                    #result.show()
            except OSError as e:
                print(e.errno)
    return results

In [10]:
def batched_prediction_test_dataset(batch_size):
    results = []
    list_of_images = get_test_images()
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    #list_of_images = ['/home/acarlier/code/test/2021-06-11-13-44-43.jpg']
    # Load the TFLite model
    print(len(list_of_images),' to predict')
    for i in range(0, len(list_of_images), batch_size):
        
        images = list_of_images[i*batch_size:(i+1)*batch_size]

        DETECTION_THRESHOLD = 0.3 #@param {type:"number"}
        detection_result_image, result = run_odt_and_draw_results(
            images,
            interpreter,
            threshold=DETECTION_THRESHOLD
        )
        results.append(result)

            
    return results

In [8]:
results_predictions = prediction_test_dataset()
#lts_predictions = prediction_test_dataset()
pickle.dump(results_predictions, open( "/home/acarlier/code/OrnithoScope/saved_models/results.p", "wb" ) )
#results_predictions = pickle.load( open( "/home/acarlier/code/OrnithoScope/saved_models/results.p", "rb" ) )
classes = get_classes_from_input()
print(len(results_predictions))
print(classes)
results_predictions

3516  to predict
0
interpreter


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2022-03-22 17:40:46.162610: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-22 17:40:48.369337: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9977 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:03:00.0, compute capability: 6.1
2022-03-22 17:40:48.372187: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10406 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:a1:00.0, compute capability: 6.1


tf.Tensor([  1 384 384   3], shape=(4,), dtype=int32)
preprocessing
avant modèle
après modèle
detection
1
interpreter
tf.Tensor([  1 384 384   3], shape=(4,), dtype=int32)
preprocessing
avant modèle
après modèle
detection
2
interpreter
tf.Tensor([  1 384 384   3], shape=(4,), dtype=int32)
preprocessing
avant modèle
après modèle
detection
3
interpreter
tf.Tensor([  1 384 384   3], shape=(4,), dtype=int32)
preprocessing
avant modèle
après modèle
detection
4
interpreter
tf.Tensor([  1 384 384   3], shape=(4,), dtype=int32)
preprocessing
avant modèle
après modèle
detection
5
interpreter
tf.Tensor([  1 384 384   3], shape=(4,), dtype=int32)
preprocessing
avant modèle
après modèle
detection
6
interpreter
tf.Tensor([  1 384 384   3], shape=(4,), dtype=int32)
preprocessing
avant modèle
après modèle
detection
7
interpreter
tf.Tensor([  1 384 384   3], shape=(4,), dtype=int32)
preprocessing
avant modèle
après modèle
detection
8
interpreter
tf.Tensor([  1 384 384   3], shape=(4,), dtype=int32)
pr

KeyboardInterrupt: 

# Préparation pour le calcul des precision recall
Il faut convertir les id de label en nom de label - DONE </br>
Il faut regrouper les label selon les file path pour avoir une liste de labels par file_path </br>
Ensuite comparaison de la liste des labels prédits et la liste des labels de vérité terrain

## Conversion de l'id en nom

In [16]:
def convert_id_to_label_name(list_of_results,list_of_classes):
    results = copy.deepcopy(list_of_results)
    for i in range(len(results)):
        for result in results[i]:
            id = int(result['class_id'])
            result['class_id'] = list_of_classes[id]
    return results

In [17]:
results_converted = convert_id_to_label_name(results_predictions, classes)

In [18]:
print(classes)
print("Before ",results_predictions[:2])
print("After ",results_converted[:4])

['MESCHA', 'SITTOR', 'MESBLE', 'MESNON', 'PINARB', 'ACCMOU', 'ROUGOR', 'VEREUR', 'MOIDOM', 'TOUTUR', 'ECUROU', 'PIEBAV', 'MULGRI', 'CAMPAG', 'MESNOI', 'MESHUP']
Before  [[{'bounding_box': array([0.09528723, 0.25329775, 0.69151413, 0.4730904 ], dtype=float32), 'class_id': 7.0, 'score': 0.66796875}], [{'bounding_box': array([0.4092438, 0.5277908, 1.0296512, 0.7281122], dtype=float32), 'class_id': 0.0, 'score': 0.375}]]
After  [[{'bounding_box': array([0.09528723, 0.25329775, 0.69151413, 0.4730904 ], dtype=float32), 'class_id': 'VEREUR', 'score': 0.66796875}], [{'bounding_box': array([0.4092438, 0.5277908, 1.0296512, 0.7281122], dtype=float32), 'class_id': 'MESCHA', 'score': 0.375}], [{'bounding_box': array([0.5509056, 0.5129066, 0.9888495, 0.9870934], dtype=float32), 'class_id': 'MESCHA', 'score': 0.9296875}], [{'bounding_box': array([0.39843464, 0.30333823, 0.6451436 , 0.5669474 ], dtype=float32), 'class_id': 'VEREUR', 'score': 0.84765625}]]


In [19]:
def get_list_of_label_from_prediction(list_of_results):
    results = copy.deepcopy(list_of_results)
    l_final = []
    for i in range(len(results)):
        l = []
        for result in results[i]:
            label = result['class_id']
            l.append(label)
        l_final.append(l)
    return l_final

In [20]:
list_of_label_from_prediction = get_list_of_label_from_prediction(results_converted)
print(list_of_label_from_prediction)

[['VEREUR'], ['MESCHA'], ['MESCHA'], ['VEREUR'], [], ['MESCHA'], ['VEREUR', 'MESCHA'], ['MESCHA'], ['MESCHA'], ['MESCHA']]


## groupby sur le DF

In [21]:
df_input = pd.read_csv('/home/acarlier/code/OrnithoScope/data/input.csv', names = columns_names)
df_input_test = df_input[df_input["split_value"] == 'TEST']
print(df_input_test.shape)
print(len(df_input_test['file_path'].unique()))
print(df_input_test['file_path'].head(10))
print('columns ',df_input_test.columns)

(3771, 10)
3516
2124    /home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_2021-03-01_09/20210301-090002_(13.0).jpg
2125     /home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_2021-03-01_09/20210301-090008_(9.0).jpg
2126    /home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_2021-03-01_09/20210301-090012_(23.0).jpg
2127    /home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_2021-03-01_09/20210301-090017_(10.0).jpg
2128    /home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_2021-03-01_09/20210301-090017_(10.0).jpg
2129    /home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_2021-03-01_09/20210301-090021_(14.0).jpg
2130    /home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_2021-03-01_09/20210301-090021_(14.0).jpg
2131     /home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_2021-03-01_09/20210301-090037_(6.0).jpg
2132     /home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_2021-03-01_09/20210301-090043_(7.0).jpg
2133     /home/acarlier/Ornith

In [22]:
def df_groupby_filepath(df_input_test):
    df_test = copy.deepcopy(df_input_test)
    df_test = df_test[['file_path','label']]
    print(df_test.columns)
    #df_test = df_test.groupby('file_path')
    #df_test.groupby('file_path')['label'].apply(lambda x: "[%s]" % ', '.join(x))
    df = df_test.groupby(['file_path'], sort=False)['label'].apply(lambda x: list(x)).to_frame().reset_index()
    #"%s" % ', '.join(x)
    print(df.columns)
    return df

In [23]:
df_groupedby = df_groupby_filepath(df_input_test)

Index(['file_path', 'label'], dtype='object')
Index(['file_path', 'label'], dtype='object')


In [24]:
print(df_groupedby.shape[0])
df_groupedby.head(10)

3516


,file_path,label
0,/home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_2021-03-01_09/20210301-090002_(13.0).jpg,[VEREUR]
1,/home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_2021-03-01_09/20210301-090008_(9.0).jpg,[MESCHA]
2,/home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_2021-03-01_09/20210301-090012_(23.0).jpg,[MESCHA]
3,/home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_2021-03-01_09/20210301-090017_(10.0).jpg,"[MESCHA, VEREUR]"
4,/home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_2021-03-01_09/20210301-090021_(14.0).jpg,"[MESCHA, MESCHA]"
5,/home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_2021-03-01_09/20210301-090037_(6.0).jpg,[MESCHA]
6,/home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_2021-03-01_09/20210301-090043_(7.0).jpg,[VEREUR]
7,/home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_2021-03-01_09/20210301-090105_(5.0).jpg,[MESCHA]
8,/home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_2021-03-01_09/20210301-090115_(7.0).jpg,[MESCHA]
9,/home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_2021-03-01_09/20210301-090133_(8.0).jpg,[MESCHA]


# Calcul des precisions recall par classes

In [25]:
def get_metrics_by_comparing_two_lists(list1,list2):
    # cas où les listes sont égales
    metrics = {'TP':0, 'FP':0, 'FN':0}
    if list1 == list2:
        metrics['TP'] += len(list1)
    else:
        for element in list1:
            if element in list2:
                metrics['TP'] += 1
            elif element not in list2:
                metrics['FP'] += 1
        for element in list2:
            if element not in list1:
                metrics['FN'] += 1
    return metrics

In [26]:
def evaluate_multiple_detections(true_labels, pred_labels):
  TP = []
  FP = pred_labels.copy()
  FN = true_labels.copy()

  for pl in pred_labels:
    if pl in true_labels:
      FP.remove(pl)
      FN.remove(pl)
      TP.append(pl)

  return TP, FN, FP

In [27]:
# première liste = prédiction et seconde liste = vérité terrain
test0 = evaluate_multiple_detections(['V'],['V']) #TP = 1
test1 = evaluate_multiple_detections(['V'],['M']) #FP = 1, FN = 1
test2 = evaluate_multiple_detections(['M'],['V']) #FN = 1, FP = 1

test0_2 = evaluate_multiple_detections(['M','V'],['M','V']) #TP = 2
test1_2 = evaluate_multiple_detections(['V', 'M'],['M','V']) #TP = 2
test2_2 = evaluate_multiple_detections(['M','I'],['V','I']) #TP = 1, FP = 

print(f"\n 1 element : \n {test0} \n {test1} \n {test2} \n 2 elements : \n {test0_2} \n {test1_2} \n {test2_2}")


 1 element : 
 (['V'], [], []) 
 ([], ['V'], ['M']) 
 ([], ['M'], ['V']) 
 2 elements : 
 (['M', 'V'], [], []) 
 (['M', 'V'], [], []) 
 (['I'], ['M'], ['V'])


In [38]:
def get_precision_recall_from_prediction(list_of_results, df_input_test, list_of_classes):
    class_metrics = []
    for classes in list_of_classes:
        class_metrics.append({'TP':0, 'FP':0, 'FN':0})
    
    for i in range(len(list_of_results)):
        pred_labels = list_of_results[i]
        true_labels = df_input_test.iloc[i]['label']
        TP, FN, FP = evaluate_multiple_detections(true_labels, pred_labels)
        print(TP, FN, FP)
        
        for lab in TP:
            class_metrics[list_of_classes.index(lab)]['TP'] += 1
        
        for lab in FN:
            class_metrics[list_of_classes.index(lab)]['FN'] += 1
            
        for lab in FP:
            class_metrics[list_of_classes.index(lab)]['FP'] += 1    
     
    class_res = []  
    for i in range(len(list_of_classes)):
        P = class_metrics[i]['TP'] / max(class_metrics[i]['TP'] + class_metrics[i]['FP'], 1)
        R = class_metrics[i]['TP'] / max(class_metrics[i]['TP'] + class_metrics[i]['FN'], 1)
        if P == 0 and R == 0:
            F_score = 0
        else:
            F_score = 2*P*R/(P+R)

        class_res.append({'Precision': P, 'Rappel': R, 'F-score': F_score})

    #accuracy = 100*total_correct_detections/(total_correct_detections + toal_incorrect_detections)
    

    return class_metrics

In [39]:
class_metrics = get_precision_recall_from_prediction(list_of_label_from_prediction, df_groupedby, classes)
class_metrics

['VEREUR'] [] []
['MESCHA'] [] []
['MESCHA'] [] []
['VEREUR'] ['MESCHA'] []
[] ['MESCHA', 'MESCHA'] []
['MESCHA'] [] []
['VEREUR'] [] ['MESCHA']
['MESCHA'] [] []
['MESCHA'] [] []
['MESCHA'] [] []


[{'TP': 6, 'FP': 1, 'FN': 3},
 {'TP': 0, 'FP': 0, 'FN': 0},
 {'TP': 0, 'FP': 0, 'FN': 0},
 {'TP': 0, 'FP': 0, 'FN': 0},
 {'TP': 0, 'FP': 0, 'FN': 0},
 {'TP': 0, 'FP': 0, 'FN': 0},
 {'TP': 0, 'FP': 0, 'FN': 0},
 {'TP': 3, 'FP': 0, 'FN': 0},
 {'TP': 0, 'FP': 0, 'FN': 0},
 {'TP': 0, 'FP': 0, 'FN': 0},
 {'TP': 0, 'FP': 0, 'FN': 0},
 {'TP': 0, 'FP': 0, 'FN': 0},
 {'TP': 0, 'FP': 0, 'FN': 0},
 {'TP': 0, 'FP': 0, 'FN': 0},
 {'TP': 0, 'FP': 0, 'FN': 0},
 {'TP': 0, 'FP': 0, 'FN': 0}]

In [16]:
def print_results(class_metrics, list_of_species):
    for i in range(0, len(class_metrics)):
        TP = class_metrics[i]['TP']
        FP = class_metrics[i]['FP']
        FN = class_metrics[i]['FN']
        print(f"{list_of_species[i]} - TP : {TP} FP : {FP} FN : {FN}")

In [17]:
print_results(class_metrics, get_classes_from_input())

MESCHA - TP : 0 FP : 0 FN : 0
SITTOR - TP : 0 FP : 0 FN : 0
MESBLE - TP : 0 FP : 0 FN : 0
MESNON - TP : 0 FP : 0 FN : 0
PINARB - TP : 0 FP : 0 FN : 0
ACCMOU - TP : 0 FP : 0 FN : 0
ROUGOR - TP : 0 FP : 0 FN : 0
VEREUR - TP : 0 FP : 0 FN : 0
MOIDOM - TP : 0 FP : 0 FN : 0
TOUTUR - TP : 0 FP : 0 FN : 0
ECUROU - TP : 0 FP : 0 FN : 0
PIEBAV - TP : 0 FP : 0 FN : 0
MULGRI - TP : 0 FP : 0 FN : 0
CAMPAG - TP : 0 FP : 0 FN : 0
MESNOI - TP : 0 FP : 0 FN : 0
MESHUP - TP : 0 FP : 0 FN : 0
